# 1. Imports

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [55]:
df = pd.read_csv('../data/raw/total dataset with los and outsid time.csv', encoding='latin1')
df.head(2)

,data_internacao_real,data_cadastro,sexo,data_nascimento,uf,municipio,profissao_ocupacao,cod_paciente,unidade_cadastro,idade_aprox,...,total_repeat_cod,total_repeat,sum_los_hour,sum_outside_hour,cumsum,cumsum.1,cumsum.1(start),cod_paciente.1,start,stop
0,7/12/2012 17:37,NaN,F,7/30/1957,SP,Ribeirão Preto,DO LAR,39,43,NaN,...,0.000000,0.0000,1943.383333,0.0,1943.383333,1943.383333,0.0,39,0.0,1943.383333
1,6/25/2012 12:00,8/2/2012 0:00,F,2/10/1971,SP,Ribeirão Preto,DESEMPREGADA,40,7,NaN,...,250.481771,6011.5625,16268.263890,48092.5,1923.150000,0.000000,0.0,40,0.0,1923.150000


# 2. Dataprep

In [57]:
cols_to_drop = [
    'unidade_cadastro',
    'cod_regulacao',
    'cod_unidade_origem',
    'tipo_regulacao',
    'data_lista_reg',
    'data_vaga_reg',
    'data_admissao_reg',
    'data_lista_hospital',
    'data_vaga_hospital',
    'data_internacao',
    'data_cancel_solicitacao',
    'data_transferencia',
    'gravidez',
    'tipo_internacao_de',
    'data_expiracao',
    'cod_transf_alta',
    'nome_unidade_solicitacao',
    'nome_unidade_destino',
    'via_internacao',
    'idade',
    'faixa_etaria'    
]

df = df.drop(cols_to_drop, axis=1)


In [4]:
df['data_internacao_real'] = pd.to_datetime(df['data_internacao_real'], format='%m/%d/%Y %H:%M', errors='raise')
df.head(2)

,data_internacao_real,data_cadastro,sexo,data_nascimento,uf,municipio,profissao_ocupacao,cod_paciente,unidade_cadastro,idade_aprox,...,total_repeat_cod,total_repeat,sum_los_hour,sum_outside_hour,cumsum,cumsum.1,cumsum.1(start),cod_paciente.1,start,stop
0,2012-07-12 17:37:00,NaN,F,7/30/1957,SP,Ribeirão Preto,DO LAR,39,43,NaN,...,0.000000,0.0000,1943.383333,0.0,1943.383333,1943.383333,0.0,39,0.0,1943.383333
1,2012-06-25 12:00:00,8/2/2012 0:00,F,2/10/1971,SP,Ribeirão Preto,DESEMPREGADA,40,7,NaN,...,250.481771,6011.5625,16268.263890,48092.5,1923.150000,0.000000,0.0,40,0.0,1923.150000


In [5]:
df['data_cadastro'] = pd.to_datetime(df['data_cadastro'], format='%m/%d/%Y %H:%M', errors='raise')
df.head(2)

,data_internacao_real,data_cadastro,sexo,data_nascimento,uf,municipio,profissao_ocupacao,cod_paciente,unidade_cadastro,idade_aprox,...,total_repeat_cod,total_repeat,sum_los_hour,sum_outside_hour,cumsum,cumsum.1,cumsum.1(start),cod_paciente.1,start,stop
0,2012-07-12 17:37:00,NaT,F,7/30/1957,SP,Ribeirão Preto,DO LAR,39,43,NaN,...,0.000000,0.0000,1943.383333,0.0,1943.383333,1943.383333,0.0,39,0.0,1943.383333
1,2012-06-25 12:00:00,2012-08-02,F,2/10/1971,SP,Ribeirão Preto,DESEMPREGADA,40,7,NaN,...,250.481771,6011.5625,16268.263890,48092.5,1923.150000,0.000000,0.0,40,0.0,1923.150000


In [6]:
df['sexo'].value_counts(dropna=False)

M    5283
F    3458
T      14
Name: sexo, dtype: int64

In [7]:
date_pattern = "(\d{1,2}/\d{1,2}/\d{4})"
df['data_nascimento'] = pd.to_datetime(df['data_nascimento'].str.extract(date_pattern).to_numpy().flat, format='%m/%d/%Y')
df.head(2)

,data_internacao_real,data_cadastro,sexo,data_nascimento,uf,municipio,profissao_ocupacao,cod_paciente,unidade_cadastro,idade_aprox,...,total_repeat_cod,total_repeat,sum_los_hour,sum_outside_hour,cumsum,cumsum.1,cumsum.1(start),cod_paciente.1,start,stop
0,2012-07-12 17:37:00,NaT,F,1957-07-30,SP,Ribeirão Preto,DO LAR,39,43,NaN,...,0.000000,0.0000,1943.383333,0.0,1943.383333,1943.383333,0.0,39,0.0,1943.383333
1,2012-06-25 12:00:00,2012-08-02,F,1971-02-10,SP,Ribeirão Preto,DESEMPREGADA,40,7,NaN,...,250.481771,6011.5625,16268.263890,48092.5,1923.150000,0.000000,0.0,40,0.0,1923.150000


In [8]:
df['uf'].value_counts(dropna=False)

SP               7039
NÃO INFORMADO    1682
MG                 23
PR                  4
BA                  3
RN                  1
AL                  1
GO                  1
RS                  1
Name: uf, dtype: int64

In [9]:
df['municipio'].value_counts(dropna=False)

Ribeirão Preto                 2724
NÃO INFORMADO                  1918
Sertãozinho                     378
MONTE ALTO                      364
JABOTICABAL                     351
                               ... 
RIO CLARO                         1
Matão                             1
BERILO                            1
São João Batista do Glória        1
MINAS NOVAS                       1
Name: municipio, Length: 108, dtype: int64

In [10]:
df['profissao_ocupacao'].value_counts(dropna=False)

NÃO INFORMADO          4834
DESEMPREGADO            535
INATIVO                 446
DO LAR                  431
ESTUDANTE               261
                       ... 
PEAO                      1
PINTOR SE SCS             1
DENTISTA                  1
AUXILIAR DE COSTURA       1
INATVA                    1
Name: profissao_ocupacao, Length: 383, dtype: int64

In [11]:
df['cod_paciente'].value_counts(dropna=False)

1068     20
230      18
1426     17
1157     16
108      15
         ..
4470      1
4472      1
4473      1
4478      1
11078     1
Name: cod_paciente, Length: 4973, dtype: int64

In [12]:
df['estado_civil'].value_counts(dropna=False)

SOLTEIRO         3547
NÃO INFORMADO    3188
CASADO           1205
SEPARADO          452
DIVORCIADO        234
viasvo            129
Name: estado_civil, dtype: int64

In [13]:
df['etnia'].value_counts(dropna=False)

NÃO INFORMADO    3578
BRANCO           3232
PARDO            1115
NEGRO             794
AMARELO            36
Name: etnia, dtype: int64

In [14]:
df['arranjo_domiciliar'].value_counts(dropna=False)

NÃO INFORMADO            5525
COM COMPANHEIRO ( A)     1617
SOZINHO                  1434
COM PAI/MÃE               122
COM OUTROS FAMILIARES      25
COM FILHOS(AS)             20
OUTRO                      11
CONHECIDOS/AMIGOS           1
Name: arranjo_domiciliar, dtype: int64

In [15]:
df['data_solicitacao'] = pd.to_datetime(df['data_solicitacao'], format='%m/%d/%Y %H:%M', errors='raise')
df.head(2)

,data_internacao_real,data_cadastro,sexo,data_nascimento,uf,municipio,profissao_ocupacao,cod_paciente,unidade_cadastro,idade_aprox,...,total_repeat_cod,total_repeat,sum_los_hour,sum_outside_hour,cumsum,cumsum.1,cumsum.1(start),cod_paciente.1,start,stop
0,2012-07-12 17:37:00,NaT,F,1957-07-30,SP,Ribeirão Preto,DO LAR,39,43,NaN,...,0.000000,0.0000,1943.383333,0.0,1943.383333,1943.383333,0.0,39,0.0,1943.383333
1,2012-06-25 12:00:00,2012-08-02,F,1971-02-10,SP,Ribeirão Preto,DESEMPREGADA,40,7,NaN,...,250.481771,6011.5625,16268.263890,48092.5,1923.150000,0.000000,0.0,40,0.0,1923.150000


In [16]:
df['tipo_regulacao'].value_counts(dropna=False)

NO    8261
DE     494
Name: tipo_regulacao, dtype: int64

In [17]:
df['cid10_diagnostico'].value_counts(dropna=False)

F19.2    1129
F20.0    1125
F31.2     667
F10.2     521
NaN       437
         ... 
f09.0       1
G71.3       1
g71.3       1
R22.0       1
Y20         1
Name: cid10_diagnostico, Length: 371, dtype: int64

In [18]:
df['cid10_seg_diagnostico'].value_counts(dropna=False)

NaN      5171
F60.3     353
F19.2     311
F10.2     169
F20.0     151
         ... 
f40.9       1
F68         1
f03.0       1
g40._       1
X63.1       1
Name: cid10_seg_diagnostico, Length: 398, dtype: int64

In [19]:
df['diff_delay_Rank_vaga_internacao_1'].value_counts(dropna=False)

NaN          1610
0.000000      825
0.016667      535
0.033333      134
0.050000       62
             ... 
24.866667       1
45.166667       1
67.800000       1
45.133333       1
8.583333        1
Name: diff_delay_Rank_vaga_internacao_1, Length: 1759, dtype: int64

In [21]:
df['data_cancel_solicitacao'].value_counts(dropna=False)

NaN                 8363
12/3/2015 10:45        3
11/23/2015 8:00        3
12/23/2015 12:10       3
12/30/2013 9:33        2
                    ... 
11/18/2013 11:51       1
11/18/2015 15:50       1
5/14/2015 11:40        1
10/25/2013 15:52       1
12/13/2017 10:30       1
Name: data_cancel_solicitacao, Length: 371, dtype: int64

In [23]:
df['traumatismo'].value_counts(dropna=False)

NÃO              7207
NÃO INFORMADO    1356
SIM               192
Name: traumatismo, dtype: int64

In [24]:
df['prob_respiratorios'].value_counts(dropna=False)

NÃO              7208
NÃO INFORMADO    1421
SIM               126
Name: prob_respiratorios, dtype: int64

In [25]:
df['avc'].value_counts(dropna=False)

NÃO              7375
NÃO INFORMADO    1315
SIM                65
Name: avc, dtype: int64

In [26]:
df['convulsao'].value_counts(dropna=False)

NÃO              6826
NÃO INFORMADO    1462
SIM               467
Name: convulsao, dtype: int64

In [27]:
df['has'].value_counts(dropna=False)

NÃO              6525
NÃO INFORMADO    1476
SIM               754
Name: has, dtype: int64

In [29]:
df['diabetes'].value_counts(dropna=False)

NÃO              6852
NÃO INFORMADO    1489
SIM               414
Name: diabetes, dtype: int64

In [30]:
df['doenca_infecto'].value_counts(dropna=False)

NÃO              6502
NÃO INFORMADO    1989
SIM               264
Name: doenca_infecto, dtype: int64

In [33]:
df['cod_transf_alta'].value_counts(dropna=False)

NaN      8754
134.0       1
Name: cod_transf_alta, dtype: int64

In [34]:
df['data_alta'].value_counts(dropna=False)

NaN                 15
9/13/2013 17:00      8
9/14/2012 9:09       7
12/3/2012 17:00      7
12/20/2012 17:00     6
                    ..
8/9/2013 11:00       1
8/12/2014 7:10       1
2/6/2015 8:05        1
12/12/2014 8:43      1
1/5/2018 11:00       1
Name: data_alta, Length: 7756, dtype: int64

In [35]:
df['cid10_diagnostico_alta'].value_counts(dropna=False)

F19.2    1718
F20.0     805
F31.2     642
F10.2     609
F60.3     346
         ... 
g93.4       1
m32.9       1
o99.3       1
f44.4       1
f42.9       1
Name: cid10_diagnostico_alta, Length: 442, dtype: int64

In [36]:
df['cid10_seg_diagnostico_alta'].value_counts(dropna=False)

NaN      5668
F19.2     292
F60.3     229
F20.0     160
F10.2      89
         ... 
g71.3       1
G71.3       1
F45.4       1
G01         1
F30         1
Name: cid10_seg_diagnostico_alta, Length: 396, dtype: int64

In [37]:
df['tipo_alta'].value_counts(dropna=False)

CLÃNICA                7038
A pedido                 612
TransferÊncia da UCP     401
TransferÊncia            339
EVASÃO                   290
Administrativa            46
NÃO INFORMADO             15
abito                     14
Name: tipo_alta, dtype: int64

In [38]:
df['tipo_internacao'].value_counts(dropna=False)

internacao voluntria      6018
internacao involuntria    1779
internacao compulsoria     957
NÃO INFORMADO                1
Name: tipo_internacao, dtype: int64

In [42]:
df['tempo_espera_internacao'].value_counts(dropna=False)

0.000000      494
NaN           230
0.011667       10
0.019444        9
0.013611        7
             ... 
171.596667      1
102.352778      1
24.039444       1
7.204167        1
0.248333        1
Name: tempo_espera_internacao, Length: 7762, dtype: int64

In [43]:
df['tempo_espera_aceite'].value_counts(dropna=False)

0.000000     494
NaN          198
0.005556      20
0.005833      20
0.004444      20
            ... 
20.740278      1
0.711111       1
59.627778      1
0.036944       1
0.363611       1
Name: tempo_espera_aceite, Length: 7377, dtype: int64

In [53]:
df['tempo_internacao'].value_counts(dropna=False).sort_index()

0.033333         1
0.049167         1
0.050000         1
0.083333         1
0.092222         1
                ..
23470.867780     1
27310.692780     1
29056.307780     1
35611.861390     1
NaN             38
Name: tempo_internacao, Length: 8565, dtype: int64